In [ ]:
#————————————————————

# Name: Azure OpenAI Admin

# Purpose:  This notebook has useful commands to clean up or troubleshoot the Azure OpenAI environment.

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch), Alex Dean (adean@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 19.05.2024
# Python Version: 3.10.4

# If necessary, download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [1]:
# Import Python packages
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI

In [2]:
# Load required variables from env file.
load_dotenv(dotenv_path=Path("C:\\Python\\azure-openai-lab\\.venv\\.env")) #Error sometimes due to \ or \\. Try one or the other. /workspaces/azure-openai-lab/.venv/.env

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [3]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    api_version = "2024-02-15-preview",
    azure_endpoint = azure_oai_endpoint
    )

In [4]:
# Upload file into Azure OpenAI Service
path_input = r"C:\Python\azure-openai-lab\data\recipes.csv" #Change path if required

# send the csv file to the assistant purpose files
response = client.files.create(
  file=open(path_input, "rb"),
  purpose="assistants"
)
print(response)
file__id = response.id

FileObject(id='assistant-EkY8qdvtcojHcDHMuaenNbTh', bytes=2915453, created_at=1716445237, filename='recipes.csv', object='file', purpose='assistants', status='processed', status_details=None)


In [5]:
# Upload file into Azure OpenAI Service
path_input = r"C:\Python\azure-openai-lab\data\recipes-preprocessed.csv" #Change path if required

# send the csv file to the assistant purpose files
response = client.files.create(
  file=open(path_input, "rb"),
  purpose="assistants"
)
print(response)
file__id = response.id

FileObject(id='assistant-4sayMlfst6QVmG1NwYysriyo', bytes=107440, created_at=1716445246, filename='recipes-preprocessed.csv', object='file', purpose='assistants', status='processed', status_details=None)


In [4]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
training_file_path = r"C:\Python\azure-openai-lab\data\recipes-training-set.jsonl"
validation_file_path = r"C:\Python\azure-openai-lab\data\recipes-validation-set.jsonl"

training_response = client.files.create(
    file=open(training_file_path, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_path, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-95ff23aec625493d8758e938c1bec3f7
Validation file ID: file-27c02b9e34814436bc3105b2c4363950


In [4]:
# Call existing uploaded file on Azure OpenAI Service
for i in client.files.list():
    if "recipes_preprocessed" in i.filename:
        file__id = i.id
        print(i.id)

assistant-rkRgo4cBcZwAG05AnplGlfM7


In [ ]:
# Clean up environment. Delete created Azure OpenAI assistants, threads and uploaded data files
for i in client.beta.assistants.list():
    client.beta.assistants.delete(i.id)
for i in client.beta.threads.list():
    client.beta.threads.delete(i.id)
for i in client.files.list():
    client.files.delete(i.id)

In [25]:
#Test run 50 people

for i in range(50):

    # Load data
    path_input = r"C:\Python\azure-openai-lab\data\recipes_preprocessed.csv" #Change path if required

    # send the csv file to the assistant purpose files
    response = client.files.create(
    file=open(path_input, "rb"),
    purpose="assistants"
    )

    file__id = response.id

    # Create an assistant
    assistant = client.beta.assistants.create(
        name = "data analyst assistant " + str(i),
        instructions = instructions,
        tools = [{"type": "code_interpreter"}],
        model = "gpt-4-1106-preview", #You must replace this value with the deployment name for your model.
        file_ids=[file__id]
    )

    fileId = assistant.file_ids[0]

    thread = client.beta.threads.create()

    prompt = "Please execute your ACTIONS on the data stored in the xlsx file " + fileId

    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = prompt
    )

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
    )
    